# EthLisbon trading strategy

This is a trading strategy demostrated in EthLisbon hackathon.

## Strategy and backtesting parameters

Here we define all parameters that affect the backtesting outcome.

In [25]:
import pandas as pd
from tradingstrategy.timebucket import TimeBucket
from tradingstrategy.chain import ChainId

TARGET_PAIR = ("GRT", "WETH")

# The backtest only consider Ethereum mainnet
BLOCKCHAIN = ChainId.ethereum

# The backtest only considers Sushiswap v2 exchange
EXCHANGE = "sushiswap"

# Use 4h candles for backtesting
CANDLE_KIND = TimeBucket.h1

# How many USD is our play money wallet
INITIAL_CASH = 10_000

# The moving average must be above of this number for us to buy
MOVING_AVERAGE_CANDLES = 50

# How many previous candles we sample for the low close value
LOW_CANDLES = 7

# How many previous candles we sample for the high close value
HIGH_CANDLES = 7

# When do we start the backtesting - limit the candle set from the data dump from the server
BACKTESTING_BEGINS = pd.Timestamp("2021-01-01")

# When do we end backtesting
BACKTESTING_ENDS = pd.Timestamp("2021-10-01")

# If the price drops 15% we trigger a stop loss
STOP_LOSS = 0.97

## Initialising the Trading Strategy client

In [26]:
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/mikkoohtamaa/.tradingstrategy


## Creating the strategy

Here is the strategy defined for Backtrader.

In [27]:
from typing import Optional
import backtrader as bt
from backtrader import indicators

from tradingstrategy.analysis.tradehint import TradeHint, TradeHintType
from tradingstrategy.frameworks.backtrader import DEXStrategy
from backtrader import analyzers, Position


class Double7(DEXStrategy):
    """An example of double-77 strategy for DEX spot trading.

    The original description: https://www.thechartist.com.au/double-7-s-strategy/
    """

    def start(self):
        # Set up indicators used in this strategy

        # Moving average that tells us when we are in the bull market
        self.moving_average = indicators.SMA(period=MOVING_AVERAGE_CANDLES)

        # The highest close price for the N candles
        # "exit" in pine script
        self.highest = indicators.Highest(self.data.close, period=HIGH_CANDLES, plot=True, subplot=False)

        # The lowest close price for the N candles
        # "entry" in pine script
        self.lowest = indicators.Lowest(self.data.close, period=LOW_CANDLES, plot=True, subplot=False)

    def next(self):
        """Execute a decision making tick for each candle."""

        # print("Tick", self.tick)
        # print("Tick", self.tick)

        close = self.data.close[0]
        low = self.lowest[-1]
        high = self.highest[-1]
        avg = self.moving_average[0]

        if not all([close, low, high, avg]):
            # Do not try to make any decision if we have nan or zero data
            return

        position: Optional[Position] = self.position

        price = close
        assert price > 0

        if not position:
            # We are not in the markets, check entry
            if close >= avg and close <= low and not position:
                # Enter when we are above moving average and the daily close was
                assert close > 0
                print(f"Tick {self.tick} buy at {price}")
                self.buy(price=price, hint=TradeHint(type=TradeHintType.open))
        else:
            # We are in the markets, check exit
            if close >= high:
                # If the price closes above its 7 day high, exit from the markets
                #print("Exited the position")
                print(f"Tick {self.tick} - exiting at {price}")
                self.close(price=price, hint=TradeHint(type=TradeHintType.close))
            else:
                # Check the exit from the market through stop loss

                # Because AMMs do not support complex order types,
                # only swaps, we do not manual stop loss here by
                # brute market sell in the case the price falls below the stop loss threshold

                entry_price = self.last_opened_buy.price
                if close <= entry_price * STOP_LOSS:
                    print(f"Tick {self.tick}, stop loss triggered. Now {close}, opened at {entry_price}")
                    self.close(price=price, hint=TradeHint(type=TradeHintType.stop_loss_triggered))


## Setting up the strategy backtest

This set ups the data sources and plumping for running the backtest ("boilerplate" in software development terms).

In [28]:
from tradingstrategy.frameworks.backtrader import prepare_candles_for_backtrader, add_dataframes_as_feeds, TradeRecorder
from tradingstrategy.pair import PandasPairUniverse

# Operate on daily candles
strategy_time_bucket = CANDLE_KIND

exchange_universe = client.fetch_exchange_universe()
columnar_pair_table = client.fetch_pair_universe()
all_pairs_dataframe = columnar_pair_table.to_pandas()
pair_universe = PandasPairUniverse(all_pairs_dataframe)

# Filter down to pairs that only trade on Sushiswap
sushi_swap = exchange_universe.get_by_name_and_chain(BLOCKCHAIN, EXCHANGE)
pair = pair_universe.get_one_pair_from_pandas_universe(
    sushi_swap.exchange_id,
    TARGET_PAIR[0],
    TARGET_PAIR[1])

assert pair, f"Could no find trading pair {TARGET_PAIR}"

all_candles = client.fetch_all_candles(strategy_time_bucket).to_pandas()
pair_candles: pd.DataFrame = all_candles.loc[all_candles["pair_id"] == pair.pair_id]
pair_candles = prepare_candles_for_backtrader(pair_candles)

# We limit candles to a specific date range to make this notebook deterministic
pair_candles = pair_candles[(pair_candles.index >= BACKTESTING_BEGINS) & (pair_candles.index <= BACKTESTING_ENDS)]

print(f"Dataset size is {len(pair_candles)} candles")

# This strategy requires data for 100 days. Because we are operating on new exchanges,
# there simply might not be enough data there
assert len(pair_candles) > MOVING_AVERAGE_CANDLES, "We do not have enough data to execute the strategy"

# Create the Backtrader back testing engine "Cebebro"
cerebro = bt.Cerebro(stdstats=True)

# Add out strategy
cerebro.addstrategy(Double7)

# Add two analyzers for the strategy performance
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(bt.analyzers.DrawDown)
cerebro.addanalyzer(TradeRecorder)

add_dataframes_as_feeds(
    cerebro,
    pair_universe,
    [pair_candles],
    BACKTESTING_BEGINS,
    BACKTESTING_ENDS,
    strategy_time_bucket,
    plot=True)

Dataset size is 3491 candles


## Running the backtest

Now when everything has been set up we execute the backtest.

In [29]:
# Run the backtest using the backtesting engine and store the results
results = cerebro.run()

Tick 2485 buy at 1.9683071374893188
Tick 2486, stop loss triggered. Now 1.8583532571792603, opened at 1.9683071374893188
Tick 2907 buy at 1.60282301902771
Tick 2983, stop loss triggered. Now 1.4768345355987549, opened at 1.60282301902771
Tick 3333 buy at 0.8004588484764099
Tick 3337, stop loss triggered. Now 0.7378554344177246, opened at 0.8004588484764099
Tick 3524 buy at 0.7570730447769165
Tick 3549, stop loss triggered. Now 0.6316179037094116, opened at 0.7570730447769165
Tick 3602 buy at 0.7081977725028992
Tick 3770 - exiting at 0.8167815208435059
Tick 3946 buy at 0.6908803582191467
Tick 4046, stop loss triggered. Now 0.6221777200698853, opened at 0.6908803582191467
Tick 4126 buy at 0.5114973783493042
Tick 4131 - exiting at 0.5403599143028259
Tick 4136 buy at 0.5068181753158569
Tick 4139, stop loss triggered. Now 0.4874907433986664, opened at 0.5068181753158569
Tick 4254 buy at 0.5383355617523193
Tick 4280 - exiting at 0.5692570805549622
Tick 4282 buy at 0.544221818447113
Tick 4307

## Analyzing the strategy results

After the strategy is run, we need to figure out how well it performs.

We use Trading Strategy toolkit to break down the trades.

In [30]:
from tradingstrategy.frameworks.backtrader import analyse_strategy_trades

strategy: Double7 = results[0]

trades = strategy.analyzers.traderecorder.trades
trade_analysis = analyse_strategy_trades(trades)


Got event AutoOrderedDict([('order', <backtrader.order.BuyOrder object at 0x7fb7b03a1130>), ('size', 1), ('price', 1.911867380142212), ('commission', 0.0)]) AutoOrderedDict([('status', 1), ('dt', 737894.5833333334), ('barlen', 0), ('size', 1), ('price', 1.911867380142212), ('value', 1.911867380142212), ('pnl', 0.0), ('pnlcomm', 0.0), ('tz', None)])
Got event AutoOrderedDict([('order', <backtrader.order.SellOrder object at 0x7fb7c4671370>), ('size', -1), ('price', 0.0), ('commission', 0.0)]) AutoOrderedDict([('status', 2), ('dt', 737894.625), ('barlen', 1), ('size', 0), ('price', 1.911867380142212), ('value', 0.0), ('pnl', -1.911867380142212), ('pnlcomm', -1.911867380142212), ('tz', None)])


AssertionError: Got invalid trade event AutoOrderedDict([('order', <backtrader.order.SellOrder object at 0x7fb7c4671370>), ('size', -1), ('price', 0.0), ('commission', 0.0)])

### Strategy key performance figures

Some standard performance figures for quantative finance.

In [ ]:
print(f"Backtesting range {BACKTESTING_BEGINS.date()} - {BACKTESTING_ENDS.date()}")
print(f"Sharpe: {strategy.analyzers.sharperatio.get_analysis()['sharperatio']:.3f}")
print(f"Normalised annual return: {strategy.analyzers.returns.get_analysis()['rnorm100']:.2f}%")
print(f"Max drawdown: {strategy.analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%")

### Trading summary

Some basic statistics on the success of trades.

In [ ]:
from IPython.core.display import HTML
from IPython.display import display

from tradingstrategy.analysis.tradeanalyzer import TradeSummary

strategy: Double7 = results[0]
cash_left = strategy.broker.get_cash()
summary: TradeSummary = trade_analysis.calculate_summary_statistics(INITIAL_CASH, cash_left)

display(HTML(summary.to_dataframe().to_html(header=False)))

### Trade success histogram

Show the distribution of won and lost trades as a histogram.

In [ ]:
from matplotlib.figure import Figure
from tradingstrategy.analysis.tradeanalyzer import expand_timeline
from tradingstrategy.analysis.profitdistribution import plot_trade_profit_distribution

# Set the colors we use to colorise our PnL.
# You can adjust there colours to make the
# trade timeline more visual.
# These colors are later used in the trade timeline table.
vmin = -0.10  # Extreme red if -15% PnL
vmax = 0.10  # Extreme green if 15% PnL

timeline = trade_analysis.create_timeline()
expanded_timeline, _ = expand_timeline(exchange_universe, pair_universe, timeline)

fig: Figure = plot_trade_profit_distribution(
    expanded_timeline,
    bins=10,
    vmin=vmin,
    vmax=vmax)

### Chart analysis

The Backtrader default output chart will display the portfolio value
develoment and when the individual trades were made.

In [ ]:
import datetime
import matplotlib.pyplot as pyplot

# Increase the size of the chart to be more readable,
# look better on high DPI monitor output
pyplot.rcParams['figure.figsize'] = [20, 10]

# We can cut the head period of the backtesting away,
# as there aren't any trades until the first moving average time period is complete
trading_begins_at = (BACKTESTING_BEGINS + MOVING_AVERAGE_CANDLES * datetime.timedelta(days=1)).date()

# Get all produced figures (one per strategy)
figs = cerebro.plot(iplot=True, start=trading_begins_at)

### Trading timeline

The timeline displays individual trades the strategy made. This is good for figuring out some really stupid trades the algorithm might have made.

In [ ]:
from tradingstrategy.analysis.tradeanalyzer import expand_timeline

# Generate raw timeline of position open and close events
timeline = trade_analysis.create_timeline()

# Because this is s a single strategy asset,
# we can reduce the columns we render in the trade summary
hidden_columns = [
    "Id",
    "PnL % raw",
    "Exchange",
    "Base asset",
    "Quote asset"
]

# Expand timeline with human-readable exchange and pair symbols
expanded_timeline, apply_styles = expand_timeline(
    exchange_universe,
    pair_universe,
    timeline,
    vmin,
    vmax,
    hidden_columns)

# Do not truncate the row output
with pd.option_context("display.max_row", None):
    display(apply_styles(expanded_timeline))

